# Analyse Spectractor Output

- adapted from https://github.com/stubbslab/PCWG-AuxTel/blob/main/Spectractor_data_access_demo.ipynb

- author : Sylvie Dagoret-campagne
- affiliation : IJCLab/IN2P3/CNRS
- update : September 2021 13th
- DM-Stack version : **w_2021_36**
- Big CPU

Make a butler, but we don't point it at the usual repo root, we point it at the location of best reduction of the data to date.
For the current version of this, see setup_script.md

In [1]:
import matplotlib.pyplot as plt
#%matplotlib inline
# correct for bug in stack in wl_2021_02
%config IPCompleter.use_jedi=False

In [2]:
#repoDir = '/project/shared/auxTel/rerun/mfl/slurmRun/'
#repoDir='/project/shared/auxTel/rerun/dagoret'
repoDir='/project/shared/auxTel/rerun/dagoret/outputspectr_scan2021_July'

import lsst.daf.persistence as dafPersist
butler = dafPersist.Butler(repoDir)

<ipython-input-2-a8e09fe1605e>:6: FutureWarning: Gen2 Butler has been deprecated (Butler). It will be removed sometime after v23.0 but no earlier than the end of 2021.
  butler = dafPersist.Butler(repoDir)
<ipython-input-2-a8e09fe1605e>:6: FutureWarning: Gen2 Butler has been deprecated (LatissMapper). It will be removed sometime after v23.0 but no earlier than the end of 2021.
  butler = dafPersist.Butler(repoDir)


Next, we make a NightReporter object. This is just a useful little toy project (read: not well supported, but might be helpful) to will help understand (and visualise) the night's observations. Try poking it to see what it does, some it it, _e.g._ reporter.printObsTable() is quite useful for getting a quick overview of the observations of the night.

In [3]:
from lsst.rapid.analysis.nightReport import NightReporter
reporter = NightReporter(repoDir, "2021-07-07")

Loaded data for seqNums 1 to 620


In [4]:
dayObs = '2021-07-07'

See what objects we looked at over the course of the selected dayObs, so that we can pick one that sampled a nice range of airmasses to examine further.

In [5]:
reporter.plotPerObjectAirMass()

In [6]:
objName = 'HD 160617'

Here we find out from the butler which sequence numbers are observations of that star. Note that a) some will be dispersed, but some will likely be direct images or donuts, etc, and that b) of the dispersed images, some might not have run all the way through the pipeline without failure, so we're going to need to test for that.

In [7]:
seqNums = butler.queryMetadata('raw', 'seqNum', dayObs=dayObs, object=objName)

Here, for the seqNums, we ask the butler to check if the output of Spectractor actually exists, such that it can be retrieved.

In [8]:
extracted = []
for seqNum in seqNums:
    if butler.datasetExists('spectraction', dayObs=dayObs, seqNum=seqNum):
        extracted.append(seqNum)

In [9]:
extracted

[234,
 235,
 236,
 237,
 239,
 238,
 241,
 240,
 242,
 243,
 244,
 247,
 248,
 249,
 260,
 261,
 262,
 263,
 266,
 267,
 268,
 269,
 272,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 292,
 293,
 298,
 299,
 300,
 301,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 352,
 353,
 354,
 355,
 359,
 361,
 362,
 365]

Grab a random single result just so we've got an object to poke at, such that we can use it to help us write the code that follows.

In [10]:
#all_results = []
#for seqNum in extracted:
#    results = butler.get('spectraction', dayObs=dayObs, seqNum=seqNum)
#    all_results.append(results)

In [11]:
idx=0

In [12]:
#print(all_results[idx].spectrum.lambdas)

In [13]:
#fig = plt.figure(figsize=(10,5))
#ax=fig.add_subplot(111)
#ax.plot(all_results[idx].spectrum.lambdas,all_results[idx].spectrum.data,'r')
#ax.set_title(title)

In [14]:
#result = butler.get('spectraction', dict(dayObs=dayObs, seqNum=extracted[0]))
result = butler.get('spectraction', dict(dayObs=dayObs, seqNum=317))

In [15]:
result.spectrum.header

SIMPLE  =                    T                                                  
TARGET  = 'HD 160617'          / name of the target in the image                
REDSHIFT= '0.000335'           / redshift of the target                         
GRATING = 'holo4_003'          / name of the disperser                          
ROTANGLE=  -0.1669043371015121 / [deg] angle of the dispersion axis             
D2CCD   =    178.5061134577691 / [mm] distance between disperser and CCD        
LSHIFT  =                  0.0                                                  
PARANGLE=    67.57804795567915 / parallactic angle in degree                    
TARGETX =    129.3446876015798 / target position on X axis                      
TARGETY =    490.8406546787762 / target position on Y axis                      
THETAFIT= -0.00051922423331379 / [deg] [USED] rotation angle from the Hessian an
THETAINT=                -0.28 / [deg] rotation angle interp from disperser scan
PSF_REG =    57999.999999999

In [16]:
title=result.spectrum.header['TARGET']

In [17]:
#result.spectrum?

In [18]:
fig = plt.figure(figsize=(10,5))
ax=fig.add_subplot(111)
ax.plot(result.spectrum.lambdas,result.spectrum.data,'r')
ax.set_title(title)
plt.show()

In [19]:
from spectractor.extractor.spectrum import *

In [20]:
fig = plt.figure(figsize=(10,5))
ax=fig.add_subplot(111)
plot_spectrum_simple(plt.gca(), result.spectrum.lambdas, result.spectrum.data, result.spectrum.err)
ax.set_title(title)

Text(0.5, 1.0, 'HD 160617')

Build a dictionary of all the results for our sequence numbers:

In [21]:
extracted

[234,
 235,
 236,
 237,
 239,
 238,
 241,
 240,
 242,
 243,
 244,
 247,
 248,
 249,
 260,
 261,
 262,
 263,
 266,
 267,
 268,
 269,
 272,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 292,
 293,
 298,
 299,
 300,
 301,
 317,
 318,
 319,
 320,
 321,
 322,
 323,
 324,
 325,
 326,
 327,
 328,
 329,
 330,
 352,
 353,
 354,
 355,
 359,
 361,
 362,
 365]

In [ ]:
results = {}
for seqNum in extracted:
    print(seqNum)
    results[seqNum] = butler.get('spectraction', dayObs=dayObs, seqNum=seqNum)

234
235
236
237
239
238
241
240
242
243
244
247
248
249
260
261
262
263
266
267
268
269
272
277
278
279
280
281
282
283
292
293
298
299
300
301
317
318
319
320
321
322
323
324
325
326
327
328
329
330
352
353
354
355


Pick a random extraction, grab its lines, and see what their names are

In [ ]:
for lineNum, line in enumerate(result.spectrum.lines.lines):
    print(lineNum, line.label)

Now that we've poked a line to see how it quacks, lets look over all the extractions, pull the O2 line out, and plot its equivalent width against airmass

In [ ]:
airmasses, eqWidths = [], []
for seqNum in results.keys():
    airmass = results[seqNum].spectrum.airmass
    line = [line for line in results[seqNum].spectrum.lines.lines if line.label=='$O_2$']
    eqWidth = line[0].fit_eqwidth_data
    airmasses.append(airmass)
    eqWidths.append(eqWidth)

In [ ]:
import matplotlib.pyplot as plt

NB: This is a garbage in, garbage out situation. The current reductions are known-bad in many places, and this is just a toy example showing how one can access data, so don't freak out that it's bad!

In [ ]:
fig = plt.figure(figsize=(10, 8))
_ = plt.plot(airmasses, eqWidths, 'or', label="Toy reduction, don't panic!")
_ = plt.xlabel('Airmass')
_ = plt.ylabel('Eq. Width O2')
_ = plt.legend(fontsize=15)
_ = plt.title('Garbage in, garbage out, calm down.')